<a href="https://colab.research.google.com/github/divaamahajan/DataMining/blob/main/Sampling_dataset_of_KitsuneNetworkAttackDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing directories

In [ ]:
!pip install -q kaggle
from google.colab import files
import os

files.upload() # Upload the kaggle.json file
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = '/root/.kaggle'

# Download the dataset using the Kaggle API
!kaggle datasets download -d ymirsky/network-attack-dataset-kitsune -p /content --unzip


Saving kaggle.json to kaggle.json
100% 18.1G/18.1G [02:52<00:00, 73.7MB/s]
100% 18.1G/18.1G [02:52<00:00, 113MB/s] 


In [ ]:
!ls /content


'Active Wiretap'   kaggle.json	   sample_data	       'SYN DoS'
'ARP MitM'	  'Mirai Botnet'  'SSDP Flood'	       'Video Injection'
 Fuzzing	  'OS Scan'	  'SSL Renegotiation'


#Importing Directories

In [70]:
import pandas as pd
import os
import random
from google.colab import files
import subprocess  # for running command-line commands

#Downloading Sampled Daatsets

##Equally distributed Datset

In [69]:
# Define the function to sample the data and save to a new CSV file
def sample_data(folder_name):
    # Set the path to the folder
    folder_path = os.path.join('/content', folder_name)

    # Search for data and label files in the folder
    data_file_path = ''
    label_file_path = ''
    for file_name in os.listdir(folder_path):
        if file_name.endswith('_dataset.csv'):
            data_file_path = os.path.join(folder_path, file_name)
        elif file_name.endswith('_labels.csv'):
            label_file_path = os.path.join(folder_path, file_name)
  
    # Load the labels into pandas dataframes
    # get the number of rows in the CSV file using the wc command in Linux/Mac
    num_rows = int(subprocess.check_output(['wc', '-l', label_file_path]).decode().split()[0])

    # read the first 5000 rows and last 5000 rows of the CSV file
    df1 = pd.read_csv(label_file_path, nrows=5000, header=0, index_col=0)
    column_names = list(df1)
    df2 = pd.read_csv(label_file_path, skiprows=num_rows-5000, nrows=5000, header=None, index_col=0)
    # set the column names of the dataframe to the values from the header row
    df2.columns = column_names
    # combine the two dataframes into one
    labels = pd.concat([df1, df2]).reset_index(drop=True)

    # Get the column index of the last column
    label_column_name = labels.columns[-1]

    # Count the records with 0 and 1 in the label column
    n_0 = labels[label_column_name].value_counts()[0]
    n_1 = labels[label_column_name].value_counts()[1]
    n = min(n_0, n_1, 1000)
    print("0 :", n_0)
    print("1 :", n_1)
    if n < 1000:
      return
    # Load the data into pandas dataframes
    # read the first 5000 rows and last 5000 rows of the CSV file
    df1 = pd.read_csv(data_file_path, nrows=5000, header=None)
    column_names = list(range(df1.shape[1]))
    df2 = pd.read_csv(data_file_path, skiprows=num_rows-5000, nrows=5000, header=None)
    # set the column names of the dataframe to the values from the header row
    df1.columns = column_names
    df2.columns = column_names

    # combine the two dataframes into one
    data = pd.concat([df1, df2]).reset_index(drop=True)

    # Merge the data and labels
    df = pd.merge(data,labels, left_index=True, right_index=True)
    
    # Drop any duplicate rows in the data and labels
    df = df.drop_duplicates()
    
    # Get the column index of the last column
    label_column_name = df.columns[-1]
    print(label_column_name)
    # Filter out records with label equal to 0 and 1
    df_0 = df[df[label_column_name] == 0]
    df_1 = df[df[label_column_name] == 1]

    # Randomly sample 1000 records from each group
    df_0_sampled = df_0.sample(n=1000, replace=True, random_state=42)
    df_1_sampled = df_1.sample(n=1000, replace=True, random_state=42)

    # Concatenate the two groups back together
    df_sampled = pd.concat([df_0_sampled, df_1_sampled]).reset_index(drop=True)

    # Shuffle the rows of the resulting dataframe
    df_sampled = df_sampled.sample(frac=1, random_state=42)

    # Download the resulting CSV file
    output_filename = f'{folder_name}_sampled.csv'
    df_sampled.to_csv(output_filename, index=False)
    files.download(output_filename)

# Call the function for each folder
folders = ['ARP MitM' , 'SSDP Flood', 'Video Injection', 'Active Wiretap', 'Fuzzing']
for folder in folders:
  print(folder, "==>")
  sample_data(folder)

SSDP Flood ==>
0 : 5018
1 : 4982
x


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##For skewed data

In [78]:
# Define the function to sample the data and save to a new CSV file
def sample_data(folder_name):
    # Set the path to the folder
    folder_path = os.path.join('/content', folder_name)

    # Search for data and label files in the folder
    data_file_path = ''
    label_file_path = ''
    for file_name in os.listdir(folder_path):
        if file_name.endswith('_dataset.csv'):
            data_file_path = os.path.join(folder_path, file_name)
        elif file_name.endswith('_labels.csv'):
            label_file_path = os.path.join(folder_path, file_name)
  
    # Load the labels into pandas dataframes
    df = pd.read_csv(label_file_path, header=0, index_col=0)
    skiprow_1 = (df[df['x'] == 1]).index[0]
    labels = pd.concat([df.iloc[:5000], df.iloc[skiprow_1:(15000+skiprow_1)]]).reset_index(drop=True)

    # Get the column index of the last column
    label_column_name = labels.columns[-1]

    # Count the records with 0 and 1 in the label column
    n_0 = labels[label_column_name].value_counts()[0]
    n_1 = labels[label_column_name].value_counts()[1]
    n = min(n_0, n_1, 1000)
    print("0 :", n_0)
    print("1 :", n_1)
    if n < 1000:
      return
    # Load the data into pandas dataframes
    # read the first 5000 rows and last 5000 rows of the CSV file
    df1 = pd.read_csv(data_file_path, nrows=5000, header=None)
    column_names = list(range(df1.shape[1]))
    df2 = pd.read_csv(data_file_path, skiprows=skiprow_1, nrows=15000, header=None)
    # set the column names of the dataframe to the values from the header row
    df1.columns = column_names
    df2.columns = column_names

    # combine the two dataframes into one
    data = pd.concat([df1, df2]).reset_index(drop=True)

    # Merge the data and labels
    df = pd.merge(data,labels, left_index=True, right_index=True)
    
    # Drop any duplicate rows in the data and labels
    df = df.drop_duplicates()
    
    # Get the column index of the last column
    label_column_name = df.columns[-1]
    print(label_column_name)
    # Filter out records with label equal to 0 and 1
    df_0 = df[df[label_column_name] == 0]
    df_1 = df[df[label_column_name] == 1]

    # Randomly sample 1000 records from each group
    df_0_sampled = df_0.sample(n=1000, replace=True, random_state=42)
    df_1_sampled = df_1.sample(n=1000, replace=True, random_state=42)

    # Concatenate the two groups back together
    df_sampled = pd.concat([df_0_sampled, df_1_sampled]).reset_index(drop=True)

    # Shuffle the rows of the resulting dataframe
    df_sampled = df_sampled.sample(frac=1, random_state=42)

    # Download the resulting CSV file
    output_filename = f'{folder_name}_sampled.csv'
    df_sampled.to_csv(output_filename, index=False)
    files.download(output_filename)

# Call the function for each folder
folders = ['OS Scan', 'SYN DoS', 'SSL Renegotiation']
for folder in folders:
  print(folder, "==>")
  sample_data(folder)
  

0 : 17129
1 : 2871
x


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Database with missing label header and index

In [80]:
# Define the function to sample the data and save to a new CSV file
def sample_data(folder_name):
    # Set the path to the folder
    folder_path = os.path.join('/content', folder_name)

    # Search for data and label files in the folder
    data_file_path = ''
    label_file_path = ''
    for file_name in os.listdir(folder_path):
        if file_name.endswith('_dataset.csv'):
            data_file_path = os.path.join(folder_path, file_name)
        elif file_name.endswith('_labels.csv'):
            label_file_path = os.path.join(folder_path, file_name)
  
  
    # get the number of rows in the CSV file using the wc command in Linux/Mac
    num_rows = int(subprocess.check_output(['wc', '-l', label_file_path]).decode().split()[0])

    # read the first 5000 rows and last 5000 rows of the CSV file
    df1 = pd.read_csv(label_file_path, nrows=5000, header=None)
    df1.columns = ['x']
    df2 = pd.read_csv(label_file_path, skiprows=num_rows-5000, nrows=5000, header=None)
    # set the column names of the dataframe to the values from the header row
    df2.columns = ['x']
    # combine the two dataframes into one
    labels = pd.concat([df1, df2]).reset_index(drop=True)

    # Get the column index of the last column
    label_column_name = labels.columns[-1]

    # Count the records with 0 and 1 in the label column
    n_0 = labels[label_column_name].value_counts()[0]
    n_1 = labels[label_column_name].value_counts()[1]
    n = min(n_0, n_1, 1000)
    print("0 :", n_0)
    print("1 :", n_1)
    if n < 1000:
      return
    # Load the data into pandas dataframes
    # read the first 5000 rows and last 5000 rows of the CSV file
    df1 = pd.read_csv(data_file_path, nrows=5000, header=None)
    column_names = list(range(df1.shape[1]))
    df2 = pd.read_csv(data_file_path, skiprows=num_rows-5000, nrows=5000, header=None)
    # set the column names of the dataframe to the values from the header row
    df1.columns = column_names
    df2.columns = column_names

    # combine the two dataframes into one
    data = pd.concat([df1, df2]).reset_index(drop=True)

    # Merge the data and labels
    df = pd.merge(data,labels, left_index=True, right_index=True)
    
    # Drop any duplicate rows in the data and labels
    df = df.drop_duplicates()
    
    # Get the column index of the last column
    label_column_name = df.columns[-1]
    print(label_column_name)
    # Filter out records with label equal to 0 and 1
    df_0 = df[df[label_column_name] == 0]
    df_1 = df[df[label_column_name] == 1]

    # Randomly sample 1000 records from each group
    df_0_sampled = df_0.sample(n=1000, replace=True, random_state=42)
    df_1_sampled = df_1.sample(n=1000, replace=True, random_state=42)

    # Concatenate the two groups back together
    df_sampled = pd.concat([df_0_sampled, df_1_sampled]).reset_index(drop=True)

    # Shuffle the rows of the resulting dataframe
    df_sampled = df_sampled.sample(frac=1, random_state=42)

    # Download the resulting CSV file
    output_filename = f'{folder_name}_sampled.csv'
    df_sampled.to_csv(output_filename, index=False)
    files.download(output_filename)

# Call the function for each folder
folders = ['Mirai Botnet']
for folder in folders:
  print(folder, "==>")
  sample_data(folder)
  

Mirai Botnet ==>
0 : 5000
1 : 5000
x


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>